<a href="https://colab.research.google.com/github/AugustvonMackensen/AI_colab/blob/main/deeplearning_mlp_bp_learning_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[1] Import Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split # split train data and test data
from tensorflow import keras
from tensorflow.keras import layers, optimizers, losses

[2] Define the functions for the data preparation

In [ ]:
def prepare_data():
  iris = load_iris()    # Read iris dataset
  X = iris.data[:, 2:]   # Select the bloom's length and width
  y = iris.target # label for each samples
  lbl_str = iris.target_names # 'setosa'. 'versicolor', 'virginica'
  X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.20) # 80% : train data, 20% : test data

  return X_tr, y_tr, X_val, y_val, lbl_str

[3] Visualize the Model

In [ ]:
def visualize(net, X, y, multi_class, labels, class_id, colors,
             xlabel, ylabel, legend_loc='lower right'):
  # list the range between min and max, the gap is 0.05
  x_max = np.ceil(np.max(X[:, 0])).astype(int)
  x_min = np.floor(np.min(X[:, 0])).astype(int)
  y_max = np.ceil(np.max(X[:, 1])).astype(int)
  y_min = np.floor(np.min(X[:, 1])).astype(int)
  x_lin = np.linspace(x_min, x_max, (x_max-x_min)*20+1)
  y_lin = np.linspace(y_min, y_max, (y_max-y_min)*20+1)

  # find x and y from the x_lin and y_Lin
  x_mesh, y_mesh = np.meshgrid(x_lin, y_lin)

  # input
  X_test = np.column_stack([x_mesh.ravel(), y_mesh.ravel()])

  # calculate output in terms of X_test
  if multi_class:
    y_hat = net.predict(X_test)
    y_hat = np.array([np.argmax(y_hat[k]) for k in range(len(y_hat))], dtype=int)
  else:
    y_hat = (net.predict(X_test) >= 0.5).astype(int)
    y_hat = y_hat.reshape(len(y_hat))

  # Set the legend and color for each classes and horizontial and vertical range
  plt.xlim(x_min, x_max)
  plt.ylim(y_min, y_max)

  # draw the Scatter Plot
  for c, i, c_name in zip(colors, labels, class_id):
    # Scatter Plot for grid coodrinate
    plt.scatter(X_test[y_hat == i, 0], X_test[y_hat == i, 1],
                c=c, s=5, alpha=0.3, edgecolors='none')
    # Scatter Plot for learning samples
    plt.scatter(X[y==i, 0], X[y==i, 1],
                c=c, s=20, label=c_name)

  # set the position for legend
  plt.legend(loc=legend_loc)
  # print graph after setting label for x-axis and y-axis
  plt.xlabel(xlabel, size=12)
  plt.ylabel(ylabel, size=12)
  plt.show()

[4] Preparing for train data

In [ ]:
nSamples = 150
nDim = 2
nClasses = 3
X_tr, y_tr, X_val, y_val, labels = prepare_data()

[5] Model

A model can be defined by using a Sequential class, which links layers that consists of the model In a sequential manner.

Model can be defined by functional API or subclassing API if it consists of a link that is complexed strcutrue.

This model has feedforward neural network.

[5] Generate BP_Model and Learning

In [ ]:
bp_model_tf = keras.Sequential() # Create a instance of Sequential class
bp_model_tf.add(layers.InputLayer(input_shape=(nDim,))) # Add Input Layer. 2 inputs are used.

# Input_shape 지정하지 않으면 학습 데이터가 제공이 될 때까지 입력 규격을 알 수 없어 가중치 등이 요소를 결정할 수 없어 모델 구성이 완성되지 않음
# 모델 구성이 완성되는 것은 첫 번째 학습 또는 모델의 출력을 구해야 하는 메소드가 실행될 때까지 지연됨
# You can directly link Dense Layer without Input Layer
# bp_model_tf.add(Dense(4, input_shape = (nDim, ), activation='sigmoid'))

bp_model_tf.add(layers.Dense(4, activation='sigmoid')) # Add Hidden Layer, which consits of 4 neuron.
bp_model_tf.add(layers.Dense(nClasses, activation='softmax')) # Add Output Layer, which consists of 3 neurons

#You can use this code
# bp_model_tf = Sequential(Dense(4, input_shape = (nDim, ), activation='sigmoid'),
#                             Dense(nClasses, activation='softmax')
#])

[6] Model Sumary information

In [ ]:
bp_model_tf.summary()

[7] Compile the Model

optimizer: set optimizer. In this code, I use SGD the learning rate is 0.1, and the momentum is 0.9
loss: set loss function(CrossEntropy)
metrics: setlist for training and validation, which evaluates accuracy for category.

In [ ]:
bp_model_tf.compile(optimizer=optimizers.SGD(0.1, momentum=0.9), loss=losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

[8] Train the model

In [ ]:
bp_model_tf.fit(X_tr, y_tr, batch_size=15, epochs=1000, verbose=2, validation_data=(X_val, y_val))

[9] Classification

line 1 : classify X_val(Validation data) by calling predict()

line 2: activation is softmax, so y_hat has 3 probability values after the classification for each input. Therefore, define the label using argmax.

line 3~4: Get accuracy by using the number of matched data(comparing calculated label to real label)


In [ ]:
y_hat = bp_model_tf.predict(X_val, verbose=0)
y_hat_lbls = np.array([np.argmax(y_hat[k]) for k in range(len(X_val))])
nCorrect = (y_hat_lbls == y_val).sum()
print('Validation accuracy : {}/{} --> {:7.3f}%'.format(nCorrect, len(X_val), nCorrect * 100.0 / len(X_val)))

[10] Visualize

In [ ]:
visualize(bp_model_tf, X_tr, y_tr, multi_class=True, class_id=labels, labels=[0, 1, 2],
          colors=['magenta', 'blue', 'green'], xlabel='petal length', ylabel='petal width')